In [1]:
import sys
sys.path.append('./models/')
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import time
import os
from data_loader import Dataset,Options
import models.unet_normals as unet
from tensorboardX import SummaryWriter
import OpenEXR, Imath

### Setup Options
Set the various parameters:
- dataroot: The folder where the training data is stored
- file_list: List of filenames of images for training
- batchSize: Batch size for model
- shuffle: If true, will shuffle the dataset
- phase: If 'train', then it's in training mode.
- num_epochs: Number of epochs to train the model for
- imsize: Dimensions of the image (square)
- num_classes: Num of classes in the output
- gpu: Which GPU device to use
- logs_path: The path where the log files (tensorboard) will be saved.

In [2]:
class OPT():
    def __init__(self):
        self.dataroot = './data/'
        self.file_list = './data/datalist'
        self.batchSize = 24
        self.shuffle = True
        self.phase = 'train'
        self.num_epochs = 1000
        self.imsize = 224
        self.num_classes = int(3)
        self.gpu = '0'
        self.logs_path = 'logs/exp4'
        self.use_pretrained = False

opt = OPT()

### Setup logging and dataloaders

In [3]:
###################### Options #############################
phase = opt.phase
device = torch.device("cuda:"+ opt.gpu if torch.cuda.is_available() else "cpu")

###################### TensorBoardX #############################
if os.path.exists(opt.logs_path):
    raise Exception('The folder \"{}\" already exists! Define a new log path or delete old contents.'.format(opt.logs_path))
    
writer = SummaryWriter(opt.logs_path, comment='create-graph')
graph_created = False

###################### DataLoader #############################
dataloader = Dataset(opt)


shuffling the dataset


### Create the model
We use a UNet model. The last few layers of this model are modified to return a 3 channel image, containing the x,y,z values of surface normal vectors.

In [4]:
###################### ModelBuilder #############################
model = unet.Unet(num_classes=opt.num_classes)

# Load weights from checkpoint
if (opt.use_pretrained == True):
    checkpoint_path = 'logs/exp2/checkpoints/checkpoint.pth'
    model.load_state_dict(torch.load(checkpoint_path))

model = model.to(device)
model.train()

###################### Setup Optimazation #############################
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
exp_lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

###################### Loss fuction #############################
'''
@input: The 2 vectors whose cosine loss is to be calculated
The dimensions of the matrices are expected to be (batchSize, 3, imsize, imsize). 

@return: 
elementwise_mean: will return the sum of all losses divided by num of elements
none: The loss will be calculated to be of size (batchSize, imsize, imsize) containing cosine loss of each pixel
'''
def loss_fn(input_vec, target_vec, reduction='elementwise_mean'):
    cos = nn.CosineSimilarity(dim=1, eps=1e-6)
    loss_val = 1.0 - cos(input_vec, target_vec)
    if (reduction=='elementwise_mean'):
        return torch.mean(loss_val)
    elif (reduction=='none'):
        return loss_val
    else:
        raise Exception('Warning! The reduction is invalid. Please use \'elementwise_mean\' or \'none\''.format())


### Train the model


In [ ]:
###################### Train Model #############################
# Calculate total iter_num
total_iter_num = 0

for epoch in range(opt.num_epochs):
    print('Epoch {}/{}'.format(epoch, opt.num_epochs - 1))
    print('-' * 10)

    # Each epoch has a training and validation phase
    running_loss = 0.0
    
    


    # Iterate over data.
    for i in range(int(dataloader.size()/opt.batchSize)):
        total_iter_num += 1
        
        # Get data
        inputs, labels =  dataloader.get_batch()
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        #ToDo: get labels into correct format
        
        ## Create Graph ##
        if graph_created == False:
            graph_created = True
            writer.add_graph(model, inputs, verbose=False)
        
        # Forward + Backward Prop
        optimizer.zero_grad()
        torch.set_grad_enabled(True)
        normal_vectors = model(inputs)
        normal_vectors_norm = nn.functional.normalize(normal_vectors, p=2, dim=1)
        
        loss = loss_fn(normal_vectors_norm, labels, reduction='elementwise_mean')
        loss.backward()
        optimizer.step()

        # statistics
        running_loss += loss.item()
        writer.add_scalar('running_loss', running_loss, total_iter_num)
        writer.add_scalar('loss', loss.item(), total_iter_num)

    epoch_loss = running_loss / (dataloader.size()/opt.batchSize)
    print('{} Loss: {:.4f}'.format(phase, epoch_loss))
    
    # Save the model checkpoint
    directory = opt.logs_path+'/checkpoints/'
    if not os.path.exists(directory):
        os.makedirs(directory)
        
    if (epoch % 25 == 0):
        filename = opt.logs_path + '/checkpoints/checkpoint-ep_{}-iter_{}.pth'.format(epoch,i)
        torch.save(model.state_dict(), filename)

# Save final Checkpoint
filename = opt.logs_path + '/checkpoints/checkpoint.pth'
torch.save(model.state_dict(), filename)


Epoch 0/999
----------


/home/shrek/anaconda3/envs/pytorch-cuda9.0/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/shrek/anaconda3/envs/pytorch-cuda9.0/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


train Loss: 1.1442
Epoch 1/999
----------
shuffling the dataset
train Loss: 0.2863
Epoch 2/999
----------
shuffling the dataset
train Loss: 0.1843
Epoch 3/999
----------
shuffling the dataset
train Loss: 0.1750
Epoch 4/999
----------
shuffling the dataset
train Loss: 0.1766
Epoch 5/999
----------
shuffling the dataset
train Loss: 0.1737
Epoch 6/999
----------
shuffling the dataset
train Loss: 0.1706
Epoch 7/999
----------
shuffling the dataset
train Loss: 0.1645
Epoch 8/999
----------
shuffling the dataset
train Loss: 0.1860
Epoch 9/999
----------
shuffling the dataset
train Loss: 0.1782
Epoch 10/999
----------
shuffling the dataset
train Loss: 0.1632
Epoch 11/999
----------
shuffling the dataset
train Loss: 0.1812
Epoch 12/999
----------
shuffling the dataset
train Loss: 0.1602
Epoch 13/999
----------
shuffling the dataset
train Loss: 0.1589
Epoch 14/999
----------
shuffling the dataset
train Loss: 0.1704
Epoch 15/999
----------
shuffling the dataset
train Loss: 0.1858
Epoch 16/999
--

shuffling the dataset
train Loss: 0.1763
Epoch 129/999
----------
shuffling the dataset
train Loss: 0.1621
Epoch 130/999
----------
shuffling the dataset
train Loss: 0.1642
Epoch 131/999
----------
shuffling the dataset
train Loss: 0.1462
Epoch 132/999
----------
shuffling the dataset
train Loss: 0.1806
Epoch 133/999
----------
shuffling the dataset
train Loss: 0.1424
Epoch 134/999
----------
shuffling the dataset
train Loss: 0.1855
Epoch 135/999
----------
shuffling the dataset
train Loss: 0.1598
Epoch 136/999
----------
shuffling the dataset
train Loss: 0.1671
Epoch 137/999
----------
shuffling the dataset
train Loss: 0.1483
Epoch 138/999
----------
shuffling the dataset
train Loss: 0.1617
Epoch 139/999
----------
shuffling the dataset
train Loss: 0.1432
Epoch 140/999
----------
shuffling the dataset
train Loss: 0.1647
Epoch 141/999
----------
shuffling the dataset
train Loss: 0.1793
Epoch 142/999
----------
shuffling the dataset
train Loss: 0.1849
Epoch 143/999
----------
shuffling 

### Benchmark the speed

Run the script below to get the estimate of fps you can achieve on your machine.
For this experiment we used ```timeit``` magic function.

In [ ]:
# %matplotlib inline

import sys, os
from PIL import Image
from matplotlib import pyplot as plt
import torch
import torch.nn as nn
from torchvision import transforms
from torch.autograd import Variable
import models.unet_normals as unet
import numpy as np
from data_loader import Dataset,Options

class OPT():
    def __init__(self):
        self.dataroot = './data/'
        self.file_list = './data/datalist'
        self.batchSize = 24
        self.shuffle = True
        self.phase = 'eval'
        self.num_epochs = 1000
        self.imsize = 224
        self.num_classes = int(3)
        self.gpu = '0'
        self.logs_path = 'logs/exp3'

opt = OPT()
dataloader = Dataset(opt)

device = torch.device("cuda:"+ opt.gpu if torch.cuda.is_available() else "cpu")

checkpoint_path = opt.logs_path + '/checkpoints/checkpoint.pth'

# exr_ground_truth_path = 'data/000000011-normals.exr'
for i in range(0, 61):
    # Open and Transform Img
    img_not_preprocessed = Image.open('data/test-imgs/%09d-rgb.jpg'%(i)).convert("RGB")
    img = dataloader.transformImage(img_not_preprocessed)
    img = img.unsqueeze(0)

    # Send img to device
    img = Variable(img.to(device))

    # Load Model
    fcn = unet.Unet(num_classes=opt.num_classes)
    fcn.load_state_dict(torch.load(checkpoint_path))
    fcn.to(device)
    fcn.eval()

    # Inference
    res = fcn(img)
    res_norm = nn.functional.normalize(res, p=2, dim=1)
    output = res_norm.squeeze(0)
    output = output.data.cpu().numpy()
    output2 = output.copy()
    output3 = output.copy()
    print('output.shape:')
    print(output.shape)

    # Display Results
    # Orig image
    plt.imshow(img_not_preprocessed)
    plt.show()

    # Output Normals visualized with RGB
    camera_normal_rgb = dataloader.normals_to_rgb(output2)
    camera_normal_rgb = np.transpose(camera_normal_rgb, (1,2,0))
    plt.imshow(camera_normal_rgb)
    plt.show()
    plt.imsave('data/test-results/%09d-normals-result-negatives-removed.png'%(i), camera_normal_rgb)

    # Output Normals visualized with RGB, negative normal values represented
    camera_normal_rgb2 = dataloader.normal_to_rgb_with_negatives(output3)
    camera_normal_rgb2 = np.transpose(camera_normal_rgb2, (1,2,0))
    plt.imshow(camera_normal_rgb2)
    plt.show()
    plt.imsave('data/test-results/%09d-normals-result.png'%(i), camera_normal_rgb)

    # Ground Truth Normals as RGB
    truth_normal = dataloader.exr_loader(exr_ground_truth_path, ndim=3)
    truth_normal_rgb = dataloader.normals_to_rgb(truth_normal)
    truth_normal_rgb = np.transpose(truth_normal_rgb, (1,2,0))
    plt.imshow(truth_normal_rgb)
    plt.show()
    plt.imsave('data/test-results/%09d-normals-groundtruth-origsize.png'%(i), truth_normal_rgb)

    # Ground Truth Normals as RGB, with negatives and resized
    truth_normal = 0
    truth_normal_rgb = 0
    truth_normal = dataloader.exr_loader('data/test-imgs/%09d-normals.exr'%(i), ndim=3)
    
    label_tensor = torch.from_numpy(truth_normal)
    label_img = transforms.ToPILImage(mode='RGB')(label_tensor)
    label_cropped = self.transformLabel(label_img)
    label_np = label_cropped.numpy()
    
    truth_normal_rgb = dataloader.normal_to_rgb_with_negatives(label_np)
    truth_normal_rgb = np.transpose(truth_normal_rgb, (1,2,0))
    plt.imshow(truth_normal_rgb)
    plt.show()
    plt.imsave('data/test-results/%09d-normals-groundtruth.png'%(i), truth_normal_rgb)